In [6]:
# ===============================
# 📘 train_bird_mouse_ann.ipynb
# ===============================

import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# -------------------------------
# ⚙️ Cấu hình
# -------------------------------
DATA_DIR = r"D:\myproject\audio_classifier\audio_classifier\dataset" 
MODEL_OUT = "bird_mouse_classifier.h5"
YAMNET_MODEL = "https://tfhub.dev/google/yamnet/1"
SAMPLE_RATE = 16000
AUDIO_DURATION = 5  # giây

# -------------------------------
# 🔹 Load YAMNet
# -------------------------------
yamnet_model = hub.load(YAMNET_MODEL)
print("✅ YAMNet model loaded!")

# -------------------------------
# 🎵 Hàm trích xuất embedding
# -------------------------------
def extract_embedding(file_path):
    try:
        # Load và resample
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE, mono=True)

        # Cắt hoặc đệm để đúng 5 giây
        target_len = SAMPLE_RATE * AUDIO_DURATION
        if len(y) > target_len:
            y = y[:target_len]
        elif len(y) < target_len:
            y = np.pad(y, (0, target_len - len(y)))

        # Trích xuất embedding YAMNet
        scores, embeddings, spectrogram = yamnet_model(y)
        return np.mean(embeddings.numpy(), axis=0)  # 1024-d vector

    except Exception as e:
        print(f"⚠️ Không đọc được file {os.path.basename(file_path)}: {e}")
        return None

# -------------------------------
# 📦 Load toàn bộ dataset
# -------------------------------
def load_dataset(base_dir):
    X, y = [], []
    labels = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    label_to_idx = {lbl: i for i, lbl in enumerate(labels)}
    print(f"📂 Classes found: {label_to_idx}")

    # Duyệt từng class
    for lbl in labels:
        folder = os.path.join(base_dir, lbl)
        for fname in os.listdir(folder):
            if not fname.lower().endswith((".wav", ".mp3", ".m4a")):
                continue
            path = os.path.join(folder, fname)
            emb = extract_embedding(path)
            if emb is not None:
                X.append(emb)
                y.append(label_to_idx[lbl])

    return np.array(X), np.array(y), label_to_idx


# -------------------------------
# 🚀 Load và chuẩn bị dữ liệu
# -------------------------------
X, y, label_map = load_dataset(DATA_DIR)
print(f"✅ Loaded {len(X)} samples")

if len(np.unique(y)) < 2 or len(X) < 3:
    raise ValueError("❌ Dataset quá ít hoặc không đủ mẫu trong từng lớp — cần >=2 file mỗi class.")

# -------------------------------
# 🧠 Train/test split
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# -------------------------------
# 🏗️ Xây ANN classifier
# -------------------------------
model = models.Sequential([
    layers.Input(shape=(1024,)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dense(len(label_map), activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# -------------------------------
# 🚀 Train model
# -------------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=8,
    verbose=1
)

# -------------------------------
# 💾 Lưu model
# -------------------------------
model.save(MODEL_OUT)
print(f"✅ Saved model to {MODEL_OUT}")

# -------------------------------
# 📈 Đánh giá nhanh
# -------------------------------
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"🎯 Test accuracy: {test_acc * 100:.2f}%")


✅ YAMNet model loaded!
📂 Classes found: {'bird': 0, 'mouse': 1, 'others': 2}
✅ Loaded 14 samples


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 295,683 (1.13 MB)

 Trainable params: 295,683 (1.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.2727 - loss: 1.1484 - val_accuracy: 0.6667 - val_loss: 0.9578
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.7273 - loss: 0.7034 - val_accuracy: 0.6667 - val_loss: 0.7834
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.7273 - loss: 0.6481 - val_accuracy: 0.6667 - val_loss: 0.6668
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8182 - loss: 0.5694 - val_accuracy: 1.0000 - val_loss: 0.5502
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9091 - loss: 0.4158 - val_accuracy: 1.0000 - val_loss: 0.4366
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9091 - loss: 0.3641 - val_accuracy: 1.0000 - val_loss: 0.3381
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9091 - loss: 0.3214 - val_accuracy: 1.0000 - val_loss: 0.2714
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 1.0000 - loss: 0.2601 - val_accuracy: 1.0000 - val_loss: 0.2279

✅ Saved model to bird_mouse_classifier.h5
🎯 Test accuracy: 100.00%
